In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import datatime

ModuleNotFoundError: No module named 'datatime'

In [6]:
plt.style.use('seaborn-v0_8-darkgrid')
mpl.rcParams['font.size'] = 16

OSError: 'seaborn-v0_8-darkgrid' not found in the style library and input is not a valid URL or path; see `style.available` for list of available styles

In [10]:
df = pd.read_csv("data/dataset_mood_smartphone.csv", index_col=0)

In [18]:
df.id.unique()
df.head()

,id,time,variable,value
1,AS14.01,2014-02-26 13:00:00.000,mood,6.0
2,AS14.01,2014-02-26 15:00:00.000,mood,6.0
3,AS14.01,2014-02-26 18:00:00.000,mood,6.0
4,AS14.01,2014-02-26 21:00:00.000,mood,7.0
5,AS14.01,2014-02-27 09:00:00.000,mood,6.0


In [ ]:
df.day =

In [17]:
plt.figure()
sns.catplot(data=df, x="time", y="value")
plt.show()


KeyboardInterrupt



<Figure size 432x288 with 0 Axes>

Error in callback <function flush_figures at 0x7fb99fa2aca0> (for post_execute):



KeyboardInterrupt

